In [7]:
#taken from https://aimatters.wordpress.com/2016/01/16/solving-xor-with-a-neural-network-in-tensorflow/

import tensorflow as tf
import time

XOR_X = [[0,0],[0,1],[1,0],[1,1]]
XOR_Y = [[0],[1],[1],[0]]

def weight_variable(shape, name):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name)


def bias_variable(shape, name):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name)


x_ = tf.placeholder(tf.float32, shape=[4,2], name = 'x-input')
y_ = tf.placeholder(tf.float32, shape=[4,1], name = 'y-input')

with tf.variable_scope('Layers'):
    Weight1 = weight_variable([2,2], name="Weight1")
    Weight2 = weight_variable([2,1], name="Weight2")

    Bias1 = bias_variable([2], name="Bias1")
    Bias2 = bias_variable([1], name="Bias2")
# Weight1 = tf.Variable(tf.random_uniform([2,2], -1, 1, seed=80636), name = "Weight1")
# Weight2 = tf.Variable(tf.random_uniform([2,1], -1, 1, seed=80636), name = "Weight2")

# Bias1 = tf.Variable(tf.zeros([2]), name = "Bias1")
# Bias2 = tf.Variable(tf.zeros([1]), name = "Bias2")

with tf.name_scope("layer2") as scope:
# 	A2 = tf.nn.relu(tf.matmul(x_, Weight1) + Bias1)
	A2 = tf.sigmoid(tf.matmul(x_, Weight1) + Bias1)


with tf.name_scope("layer3") as scope:
#     Hypothesis = tf.nn.relu(tf.matmul(A2, Weight2) + Bias2)
	Hypothesis = tf.sigmoid(tf.matmul(A2, Weight2) + Bias2)

with tf.name_scope("cost") as scope:
	cost = tf.reduce_mean(( (y_ * tf.log(Hypothesis)) + 
		((1 - y_) * tf.log(1.0 - Hypothesis)) ) * -1)

with tf.name_scope("train") as scope:
	train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
    

init = tf.global_variables_initializer()
sess = tf.Session()

writer = tf.summary.FileWriter("./logs/xor_logs/2nodes", sess.graph)

sess.run(init)

## SAVE
# saver = tf.train.Saver()

t_start = time.clock()
for i in range(50000):
#for i in range(100001):
	sess.run(train_step, feed_dict={x_: XOR_X, y_: XOR_Y})
	if i % 10000 == 0:
		print('Epoch ', i)
		print('Hypothesis ', sess.run(Hypothesis, feed_dict={x_: XOR_X, y_: XOR_Y}))
# 		print('Weight1 ', sess.run(Weight1))
# 		print('Bias1 ', sess.run(Bias1))
# 		print('Weight2 ', sess.run(Weight2))
# 		print('Bias2 ', sess.run(Bias2))
		print('cost ', sess.run(cost, feed_dict={x_: XOR_X, y_: XOR_Y}))
    
        #SAVE
#         saver.save(sess, "xor_tf/" + '/model', global_step=i + 1)

    
print('Weight1 ', sess.run(Weight1))
print('Bias1 ', sess.run(Bias1))
print('Weight2 ', sess.run(Weight2))
print('Bias2 ', sess.run(Bias2))

saver = tf.train.Saver()
saver.save(sess, "./xor.pb")

t_end = time.clock()
print('Elapsed time ', t_end - t_start)

for op in tf.get_default_graph().get_operations():
    print(op.name)

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Weight1' has type str, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Weight1' has type str, but expected one of: int, long, bool
('Epoch ', 0)
('Hypothesis ', array([[0.49992275],
       [0.49809483],
       [0.49859875],
       [0.49679565]], dtype=float32))
('cost ', 0.6931675)
('Epoch ', 10000)
('Hypothesis ', array([[0.5014072 ],
       [0.49982238],
       [0.5003042 ],
       [0.49873832]], dtype=float32))
('cost ', 0.69315845)
('Epoch ', 20000)
('Hypothesis ', array([[0.5012383 ],
       [0.49982053],
       [0.5002934 ],
       [0.4988906 ]], dtype=float32))
('cost ', 0.6931561)
('Epoch ', 30000)
('Hypothesis ', array([[0.501091  ],
       [0.49982   ],
       [0.50028515],
       [0.49902582]], dtype=float32)

In [8]:
print sess.run(Hypothesis, feed_dict={x_: XOR_X, y_: XOR_Y})

[[0.5008396 ]
 [0.4998197 ]
 [0.50027317]
 [0.49925998]]


In [9]:
freeze_var_names = list(set(v.op.name for v in tf.global_variables()))
print freeze_var_names
output_names = [Hypothesis.op.name] #['layer3/Sigmoid'] #[y_.op.name] #[model.outputs[0].name.split(":")[0]]
print output_names

from tensorflow.python.framework.graph_util import remove_training_nodes

sub_graph_def = remove_training_nodes(sess.graph_def)


from tensorflow.python.framework import graph_util

frozen_graph = graph_util.convert_variables_to_constants(sess, 
                                                         sub_graph_def, 
                                                         output_names, 
                                                         freeze_var_names)

tf.train.write_graph(frozen_graph, "models", "xor2n.pb", as_text=False)
#print "xor.pb written"

[u'Layers_3/Variable_2', u'Layers/Variable', u'Layers_3/Variable_3', u'Layers_1/Variable', u'Layers_2/Variable_2', u'Layers_3/Variable', u'Layers_3/Variable_1', u'Layers/Variable_1', u'Layers_2/Variable_3', u'Layers/Variable_3', u'Layers/Variable_2', u'Layers_2/Variable', u'Layers_4/Variable', u'Layers_4/Variable_1', u'Layers_4/Variable_2', u'Layers_4/Variable_3', u'Layers_1/Variable_3', u'Layers_1/Variable_2', u'Layers_1/Variable_1', u'Layers_2/Variable_1']
[u'layer3_3/Sigmoid']
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


'models/xor2n.pb'

In [ ]:
# utensor-cli convert models/xor2n.pb --output-nodes=layer3_3/Sigmoid
# unsupported op type in uTensor: Sigmoid

In [6]:
saver.save(sess, "./xor_tf/xor_2nodes")
out_nodes = [y_.op.name]

out_nodes = ['layer3']
print(out_nodes)


### Remove unnecessary training nodes
from tensorflow.python.framework.graph_util import remove_training_nodes

sub_graph_def = remove_training_nodes(sess.graph_def)

print sub_graph_def

###

from tensorflow.python.framework import graph_util as gu


sub_graph_def = gu.convert_variables_to_constants(sess, sub_graph_def, out_nodes)

print sub_graph_def


### save
graph_path = tf.train.write_graph(sub_graph_def,
                                  "./xor_tf/",
                                  "xor.pb",
                                  as_text=False)

print('written graph to: %s' % graph_path)


###
# sess.close()

['layer3']
node {
  name: "x-input"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 4
        }
        dim {
          size: 2
        }
      }
    }
  }
}
node {
  name: "y-input"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 4
        }
        dim {
          size: 1
        }
      }
    }
  }
}
node {
  name: "random_uniform/shape"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\002\000\000\000\002\000\000\000"
      }
    }
  }
}
node {
  name: "random_uniform/min"
  op: "Const"
  attr {
    key: "dtype"
    value {
   

AssertionError: layer3 is not in graph